In [ ]:
!pip install flask-ngrok

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, redirect, url_for, request, render_template
import numpy as np
import cv2

app = Flask(__name__, template_folder='/content/Deploy-and-Make-Predictions-with-ONNX/templates', static_folder='/content/Deploy-and-Make-Predictions-with-ONNX/static')
run_with_ngrok(app) 

@app.route('/', methods=['GET'])
def index():
    return render_template('index.html')

@app.route('/predict', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        imagefile = request.files['file']
        image_path = '/content/Deploy-and-Make-Predictions-with-ONNX/uploads/' + imagefile.filename 
        imagefile.save(image_path)
        label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'] 
        label_names.sort()
        net = cv2.dnn.readNetFromONNX('/content/Deploy-and-Make-Predictions-with-ONNX/model/cifar_classifier.onnx')
        img = cv2.imread(image_path)
        img = cv2.resize(img,(32,32))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = np.array([img]).astype('float64') / 255.0
        net.setInput(img)
        out = net.forward()
        index = np.argmax(out[0])
        label =  label_names[index]
        return label
    return None

app.run()